In [ ]:
import json
import os
import numpy as np
from pprint import pprint
from copy import deepcopy
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, auc

import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data
from torchvision.ops import box_iou, nms
from tqdm import tqdm
from collections import Counter

In [ ]:
def compute_ap(recall, precision):
    """ Compute the average precision, given the recall and precision curves.
    Code originally from https://github.com/rbgirshick/py-faster-rcnn.
    # Arguments
        recall:    The recall curve (list).
        precision: The precision curve (list).
    # Returns
        The average precision as computed in py-faster-rcnn.
    """
    # correct AP calculation
    # first append sentinel values at the end
    mrec = np.concatenate(([0.], recall, [1.]))
    mpre = np.concatenate(([0.], precision, [0.]))

    # compute the precision envelope
    for i in range(mpre.size - 1, 0, -1):
        mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

    # to calculate area under PR curve, look for points
    # where X axis (recall) changes value
    i = np.where(mrec[1:] != mrec[:-1])[0]

    # and sum (\Delta recall) * prec
    ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap

In [ ]:
anno_list = [ 592]
case_list = [2007]

In [ ]:
SEG_anno_path = '../data/N4_SEG_img/'
SEG_anno_list = sorted(os.listdir(SEG_anno_path))

In [ ]:
seg_count_dict = {}

for seg_anno in tqdm(SEG_anno_list):
    seg_anno_name = '_'.join(seg_anno.split('_')[:-1])
    if seg_anno_name not in seg_count_dict:
        seg_count_dict[seg_anno_name] = {'GT':[],
                                         'Hit':[]}
    seg_anno_array = np.load(SEG_anno_path + seg_anno)['BBOX']
    seg_anno_label_list = np.unique(seg_anno_array).tolist()
    for anno_label in seg_anno_label_list:
        if anno_label == 0:
            continue
        elif anno_label in seg_count_dict[seg_anno_name]['GT']:
            continue
        else:
            seg_count_dict[seg_anno_name]['GT'].append(anno_label)

In [ ]:
seg_label_list = []
for name in seg_count_dict:
    seg_label_list += seg_count_dict[name]['GT']
    
len(seg_label_list)

In [ ]:
len(seg_label_list)

In [ ]:
Counter(seg_label_list)

In [ ]:
model_path = '../test_output/N4_All/'
pprint([each for each in sorted(os.listdir(model_path))])

In [ ]:
key = 'mask_rcnn_FLAIR_SEG'

In [ ]:
result_path = '../test_output/N4_All/' + key + '/'
print(result_path)
result_list = sorted([each for each in os.listdir(result_path)])

In [ ]:
NMS_IOU = 0.01

result_list = sorted([each for each in os.listdir(result_path) if each[0] == 'K'])
K_pid_score = {}

for k_idx in range(0,5):
    K_pid_score[str(k_idx)] = {}
    
    tmp_result_npz = [each for each in result_list if each.startswith('K%s'%k_idx)][0]
    print(tmp_result_npz)
    tmp_result_path = result_path + tmp_result_npz
    tmp_result_file = np.load(tmp_result_path, allow_pickle=True)
    case_list = tmp_result_file['case']

    all_detections = tmp_result_file['det']
    all_annotations = tmp_result_file['anno']
    
    false_positives = np.zeros((0,))
    true_positives = np.zeros((0,))
    scores = np.zeros((0,))
    num_annotations = 0.0
    
    for i in range(len(case_list)):
        case = case_list[i]
        pid = '_'.join(case.split('_')[:-1])
        
        if pid not in K_pid_score[str(k_idx)]:
            K_pid_score[str(k_idx)][pid] = []
        
        detections = all_detections[i]
        annotations = all_annotations[i]
        num_annotations += annotations.shape[0]
        detected_annotations = []
        
        bb_scores = torch.tensor(detections[:,4])
        anchorBoxes = torch.tensor(detections[:,:4])
        anchors_nms_idx = nms(anchorBoxes, bb_scores, 0.1)
        anchors_nms_idx = anchors_nms_idx.numpy()
        detections = detections[anchors_nms_idx]
        
        for d in detections:
            det_score = d[4]
            K_pid_score[str(k_idx)][pid] += [det_score]
    
    for pid, score_list in K_pid_score[str(k_idx)].items():
        K_pid_score[str(k_idx)][pid] = np.mean(sorted(score_list)[-3:])

In [ ]:
S_IOU = 0.05
H_IOU = 0.3
NMS_IOU = 0.01

result_list = sorted([each for each in os.listdir(result_path) if each[0] == 'K'])
for k_idx in range(0,5):
    Count_dict = deepcopy(seg_count_dict)
    tmp_result_npz = [each for each in result_list if each.startswith('K%s'%k_idx)][0]
    print(tmp_result_npz)
    tmp_result_path = result_path + tmp_result_npz
    tmp_result_file = np.load(tmp_result_path, allow_pickle=True)
    case_list = tmp_result_file['case']

    all_detections = tmp_result_file['det']
    all_annotations = tmp_result_file['anno']
    
    false_positives = np.zeros((0,))
    true_positives = np.zeros((0,))
    scores = np.zeros((0,))
    num_annotations = 0.0
    
    pid_seg_tp_score = {}
    neg_seg_fp_score = {}
    neg_pid_list = []
    neg_score_list = []
    
    for i in range(len(case_list)):
        case_name = case_list[i]
        seg_anno_name = '_'.join(case_name.split('_')[:-1])
        
        if seg_anno_name not in pid_seg_tp_score:
            pid_seg_tp_score[seg_anno_name] = {}
            
        if seg_anno_name not in neg_seg_fp_score and seg_anno_name[0] == 'I':
            neg_seg_fp_score[seg_anno_name] = 0
        
        if case_name + '.npz' in SEG_anno_list:
            SEG_anno = np.load(SEG_anno_path + case_name + '.npz')['BBOX']
                
        detections = all_detections[i]
        annotations = all_annotations[i]
        num_annotations += annotations.shape[0]
        
        bb_scores = torch.tensor(detections[:,4])
        anchorBoxes = torch.tensor(detections[:,:4])
        anchors_nms_idx = nms(anchorBoxes, bb_scores, NMS_IOU)
        anchors_nms_idx = anchors_nms_idx.numpy()
        detections = detections[anchors_nms_idx]
        
        if len(detections) == 0 and annotations.shape[0] == 0 and seg_anno_name[0] == 'I':
            neg_pid_list.append(seg_anno_name)
            neg_score_list.append(0)
            continue
        
        for d in detections:
            if d[4] < S_IOU:
                continue
                
            tmp_score = d[4]  * K_pid_score[str(k_idx)][seg_anno_name]
            
            if annotations.shape[0] == 0:
                false_positives = np.append(false_positives, 1)
                true_positives = np.append(true_positives, 0)
                
                if seg_anno_name[0] == 'I':
                    neg_pid_list.append(seg_anno_name)
                    neg_score_list.append(tmp_score)
                    neg_seg_fp_score[seg_anno_name] = max(neg_seg_fp_score[seg_anno_name], tmp_score)
                continue

            d_tensor = torch.tensor(d[:4][np.newaxis])
            a_tensor = torch.tensor(annotations)
            overlaps = box_iou(d_tensor, a_tensor).numpy()
            assigned_annotation = np.argmax(overlaps, axis=1)
            max_overlap = overlaps[0, assigned_annotation]

            if max_overlap >= H_IOU:
                x1,y1,x2,y2 = a_tensor[assigned_annotation].data.numpy().astype('uint16')[0]
                x_mid = (x1+x2)//2
                y_mid = (y1+y2)//2

                hit_anno = SEG_anno[y_mid, x_mid]
                if hit_anno == 0:
                    continue
                elif hit_anno in Count_dict[seg_anno_name]['Hit']:
                    continue
                else:
                    Count_dict[seg_anno_name]['Hit'].append(hit_anno)
                    
                if seg_anno_name[0] != 'I':
                    hit_anno_name = str(hit_anno)
                    if hit_anno_name not in pid_seg_tp_score[seg_anno_name]:
                        pid_seg_tp_score[seg_anno_name][hit_anno_name] = tmp_score
                    else:
                        ori_score = pid_seg_tp_score[seg_anno_name][hit_anno_name]
                        pid_seg_tp_score[seg_anno_name][hit_anno_name] = max(ori_score, tmp_score)
            else:
                if seg_anno_name[0] == 'I':
                    neg_pid_list.append(seg_anno_name)
                    neg_score_list.append(tmp_score)
                    neg_seg_fp_score[seg_anno_name] = max(neg_seg_fp_score[seg_anno_name], tmp_score)
    
    seg_label_list = []
    hit_label_list = []
    for name in Count_dict:
        seg_label_list += Count_dict[name]['GT']
        hit_label_list += Count_dict[name]['Hit']
    
    seg_num = len(seg_label_list)
    hit_num = len(hit_label_list)
    
    pos_score_list = []
    for pid in pid_seg_tp_score:
        for hit_anno_name in pid_seg_tp_score[pid]:
            score = pid_seg_tp_score[pid][hit_anno_name]
            pos_score_list.append(score)
            
    unique_score_list = sorted(list(set(pos_score_list + neg_score_list)))
    unique_score_list.insert(0, -1)
    unique_score_list.append(1.1)
    
    unique_pid_list = np.unique(np.array(neg_pid_list).tolist())
    Neg_pid_num = len(np.unique(np.array(neg_pid_list).tolist()))
    
    sens_case_list = []
    spec_pid_list = []

    for th_score in unique_score_list:
        TP_case_num = np.sum(np.array(pos_score_list) > th_score)
        sens_case = TP_case_num / seg_num
        sens_case_list.append(sens_case)
        
        FP_pid_num = len(np.unique(np.array(neg_pid_list)[np.array(neg_score_list) > th_score]).tolist())
        spec_pid_list.append(FP_pid_num/Neg_pid_num)

#     for sens_idx, sens in enumerate(sens_case_list):
#         print(sens, spec_pid_list[sens_idx])
    

#     fig = plt.figure(figsize=(6,6),dpi=100)
#     plt.plot(spec_pid_list, sens_case_list)
#     plt.xlim(-0.1,1.1)
#     plt.ylim(-0.1,1.1)
#     plt.show()
#     break

    print(seg_num, hit_num, round(hit_num / seg_num,4))
    print('AFROC:', auc(spec_pid_list, sens_case_list).round(4))
    
    for i in sorted(list(set(seg_label_list))):
        hit_num = np.sum(np.array(hit_label_list)==i)
        seg_num = np.sum(np.array(seg_label_list)==i)
        print(i, hit_num, seg_num, (hit_num / seg_num).round(4))
        
    print('==================================')